## Setup Github

Import the github repository and move it into the local directory (also remove the defult samples directoy if it exists)

In [0]:
from getpass import getpass
import os

user = "Boyne272"
password = getpass('github password')
os.environ['GITHUB_AUTH'] = user + ':' + password

# clone the repo
!git clone https://$GITHUB_AUTH@github.com/msc-acse/acse-8-miniproject-softmax.git repo
    
# move the repo up one
!mv repo/* .
!mv repo/.git .

# delete un needed stuff
!rm -r repo
!rm -r sample_data/

github password··········
Cloning into 'repo'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 347 (delta 85), reused 102 (delta 44), pack-reused 195
Receiving objects: 100% (347/347), 38.94 MiB | 35.48 MiB/s, done.
Resolving deltas: 100% (186/186), done.
mv: cannot move 'repo/data' to './data': Directory not empty
mv: cannot move 'repo/models' to './models': Directory not empty
mv: cannot move 'repo/spec' to './spec': Directory not empty
mv: cannot move 'repo/training' to './training': Directory not empty
mv: cannot move 'repo/.git' to './.git': Directory not empty
rm: cannot remove 'sample_data/': No such file or directory


### Github Commands

In [0]:
!git checkout richard
# !git reset --hard
# !git pull origin richard
!ls -l
!git log

D	.gitignore
Branch 'richard' set up to track remote branch 'richard' from 'origin'.
Switched to a new branch 'richard'
total 36
drwxr-xr-x 2 root root 4096 May 21 15:51 data
drwxr-xr-x 2 root root 4096 May 21 15:51 models
drwxr-xr-x 2 root root 4096 May 21 16:47 __pycache__
-rw-r--r-- 1 root root  771 May 21 16:47 README.md
drwxr-xr-x 2 root root 4096 May 21 15:51 spec
-rw-r--r-- 1 root root 8812 May 21 16:47 tools.py
drwxr-xr-x 3 root root 4096 May 21 16:47 training
commit 1dc2984cfc283f50f81b4ad80c74863ea941774b (HEAD -> richard, origin/richard)
Merge: f024bd8 a252fd8
Author: Boyne272 <boynerichard@yahoo.co.uk>
Date:   Tue May 21 15:50:52 2019 +0100

    Merge branch 'gary-mao' of https://github.com/msc-acse/acse-8-miniproject-softmax into richard

commit f024bd8c9d00a0225c630d2c4bdbf7acbbd36b29
Merge: b7b3eab d0015db
Author: Boyne272 <boynerichard@yahoo.co.uk>
Date:   Tue May 21 15:50:41 2019 +0100

    Merge branch 'uni_machine' of https://github.com/msc-acse/acse-8-miniproject-so

## Setup Imports

Ipython imports and functions

In [0]:
# ipython setup
!pip install pycm livelossplot
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/usr/local/lib/python3.6/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['random', 'Normalize']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Module imports

In [0]:
# imports
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from torchvision.transforms import Compose, ToTensor, Normalize, RandomRotation, RandomCrop, ToPILImage, Pad

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

# imports from github
from tools import *

Check runtime device

In [0]:
# check running device
device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

Cuda installed! Running on GPU!


## Load Data

In [0]:
# train_feat = np.load("/content/data/kmnist-train-imgs.npy")
# train_targ = np.load("/content/data/kmnist-train-labels.npy")
test_feat = np.load("/content/data/kmnist-test-imgs.npy")

## Load Models

In [0]:
# AlexNet_half (Dropout)
class AlexNet_half_dropout(nn.Module):
    """
    Neural Network inspired on Alexnet, with a reduced number of parameters (989410).
    """

    def __init__(self, bias=True):
        "setup the neural network"
        
        # initalise
        super(AlexNet_half_dropout, self).__init__()

        # create the activation function
        act = nn.ReLU()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(1, 24, padding=1, kernel_size=4, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=1, stride=1), act,
            nn.Conv2d(24, 64, padding=2, kernel_size=5, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=3, stride=2), act,
            nn.Conv2d(64, 96, padding=2, kernel_size=5, stride=1, bias=bias), act,
            nn.Conv2d(96, 96, padding=1, kernel_size=3, stride=1, bias=bias), act,
            nn.Conv2d(96, 64, padding=1, kernel_size=3, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=3, stride=2), act)
        
        self.full_connected = nn.Sequential(
            nn.Dropout(),
            nn.Linear(2304, 256, bias=bias), act,
            nn.Dropout(),
            nn.Linear(256, 256, bias=bias), act,
            nn.Linear(256, 10, bias=bias)
        )

    def flatten(self, T):
        "flatten the image for the fully connected layers"
        return T.view(-1, T.size(1)*T.size(2)*T.size(3))
        
        
    def forward(self, x):
        "Pass through the neural network"
        x = self.convolutional(x)
        x = self.flatten(x)
        x = self.full_connected(x)
        return x

In [0]:
# AlexNet_half (Dropout)
class AlexNet_half(nn.Module):
    """
    Neural Network inspired on Alexnet, with a reduced number of parameters (989410).
    """

    def __init__(self, bias=True):
        "setup the neural network"
        
        # initalise
        super(AlexNet_half_dropout, self).__init__()

        # create the activation function
        act = nn.ReLU()
        
        self.convolutional = nn.Sequential(
            nn.Conv2d(1, 24, padding=1, kernel_size=4, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=1, stride=1), act,
            nn.Conv2d(24, 64, padding=2, kernel_size=5, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=3, stride=2), act,
            nn.Conv2d(64, 96, padding=2, kernel_size=5, stride=1, bias=bias), act,
            nn.Conv2d(96, 96, padding=1, kernel_size=3, stride=1, bias=bias), act,
            nn.Conv2d(96, 64, padding=1, kernel_size=3, stride=1, bias=bias), act,
            nn.MaxPool2d(kernel_size=3, stride=2), act)
        
        self.full_connected = nn.Sequential(
            nn.Linear(2304, 256, bias=bias), act,
            nn.Linear(256, 256, bias=bias), act,
            nn.Linear(256, 10, bias=bias)
        )

    def flatten(self, T):
        "flatten the image for the fully connected layers"
        return T.view(-1, T.size(1)*T.size(2)*T.size(3))
        
        
    def forward(self, x):
        "Pass through the neural network"
        x = self.convolutional(x)
        x = self.flatten(x)
        x = self.full_connected(x)
        return x

In [0]:
dropout = torch.load("/content/models/AlexNet_drop_60.pth")
augment = torch.load("/content/models/AlexNet_CropRot_60epoch.pth")
heavy = torch.load("/content/models/AlexNet_HeavyAug_80epoch.pth")

In [0]:
log_pdfs = []

for dictonary in [dropout, augment, heavy]:
    wrapper = train_wrapper(dictonary["model"], None, None, None, device=device)
    wrapper.transform = dictonary["transform"]
    log_pdf = wrapper.evaluate(test_feat)
    log_pdfs.append(log_pdf)

No transform found, test data must be normalised manually
No transform found, test data must be normalised manually
No transform found, test data must be normalised manually


In [0]:
log_pdfs = np.stack(log_pdfs)
print(log_pdfs.shape)
pdfs = np.exp(log_pdfs)

In [0]:
# sanity check
pdfs.sum(axis=2)

array([[0.99999964, 1.0000005 , 1.        , ..., 1.        , 0.99999964,
        0.9999999 ],
       [1.0000004 , 1.0000002 , 1.        , ..., 1.0000004 , 1.0000002 ,
        1.0000004 ],
       [0.99999994, 1.        , 1.        , ..., 0.9999994 , 1.0000005 ,
        1.0000006 ]], dtype=float32)

In [0]:
avg_pdf = pdfs.mean(axis=0)
avg_pdf.shape

(10000, 10)

In [0]:
prdict = avg_pdf.argmax(axis=1)
prdict

array([2, 9, 3, ..., 9, 4, 2])

In [0]:
save_csv(prdict, "combined_prediction", path="/")

successfully saved in /combined_prediction.csv
